# wps_climdex_fd


WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf) function, [climdex.fu](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=8)

Takes a climdexInput object as input and computes the FD (frost days) climdexindex:  that is,  the annual count of days when daily minimum temperature is below 0 degrees Celsius

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.testing import get_target_url, local_path

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
quail.climdex_fd?

Signature:
quail.climdex_fd(
    ci_file,
    ci_name=None,
    loglevel='INFO',
    output_obj='fd',
    output_file='output.rda',
)
Docstring:
Computes the annual count of days when daily minimum temperature is below 0 degrees Celsius

Parameters
----------
ci_file : ComplexData:mimetype:`application/x-gzip`
    File that holds climdexInput object (recommended file extension .rda)
ci_name : string
    Name of the climdexInput obejct
output_obj : string
    Name of the output object
output_file : string
    Filename to store the output (recommended file extension .rda)
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
frost_days_file : ComplexData:mimetype:`application/x-gzip`
    A vector containing the number of frost days for each year
File:      ~/Desktop/quail/</home/sangwonl/.local/lib/python3.6/site-packages/birdy/client/base.py-0>
Type:      method


#### Run wps_climdex_fd process with Rdata file

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="frost_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_fd(
            ci_file=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",
            output_obj="fd",
            output_file=output_file.name,
        )

#### Test output against expected output

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    urlretrieve(output.get()[0], tmp_file.name)
    robjects.r(f"load(file='{tmp_file.name}')")
    output_vector = robjects.r("fd")
    
robjects.r("load(file='{}')".format(resource_filename("tests","data/frost_days.rda")))
expected_vector = robjects.r("fd")
    
for index in range(len(expected_vector)):
    assert str(output_vector[index]) == str(expected_vector[index])